In [5]:
# Demo for visualisation of crop type and yield data
import warnings
import numpy as np
from pyproj import Transformer
import rasterio as rio
import os

# 3D stuff
from IPython.core.display import display
import json

import geopandas as gpd
import pandas as pd

from shapely.geometry import Point  # Point class
from sklearn import preprocessing

from sklearn.preprocessing import StandardScaler
from tqdm.auto import tqdm
from sklearn.model_selection import cross_validate, cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import ShuffleSplit
import seaborn as sns

sns.set_theme()
sns.set_style("darkgrid")

warnings.filterwarnings("ignore")

tqdm.pandas()

In [6]:
fields = [
    'Baumacker', 'D8', 'Dichtlacker', 'Heindlacker', 'Heng', 'Holzacker',
    'Neulandsiedlung', 'Itzling2', 'Itzling5', 'Itzling6', 'Schluetterfabrik',
    'Thalhausen138', 'Thalhausen141', 'Voettingerfeld'
]

# fields =  ['Dichtlacker', 'Heindlacker', 'Heng',
#                            'Holzacker', 'Neulandsiedlung','Itzling5',
#                            'Itzling6', 'Schluetterfabrik', 'Thalhausen138', 'Voettingerfeld']

test_fields = ['Baumacker', 'Itzling2', 'Thalhausen141']

field_summary = pd.read_excel(
    "../../data/cropdata/Bavaria/yields/fields_summary.xlsx")
yields_2018 = pd.read_csv("../../data/cropdata/Bavaria/yields/yields2018.csv")
yields_df = yields_2018.copy()
yields_df2 = yields_2018.copy()

bands = ["B04", "B05", "B06", "B07", "B08", "B8A", "B09", "B11", "B12"]
angles = ['solar_zenith', 'observer_zenith', 'relative_azimuth']
other_features = ["et0", "rain", "cum_rain"]
feature_cols = bands + other_features
target_col = "Ertr.masse (Nass)(tonne/ha)"

In [7]:
fields = yields_df.Name.unique().tolist()

In [8]:
conversion = 1


def getYieldwithoutBorders(group):
    # print(group['Name'].values[0])
    _fieldname = group['Name'].values[0]
    geo_df = gpd.GeoDataFrame.from_file(
        '../../data/cropdata/Bavaria/yields/FeldstueckeTUM/Feldstuecke_WGS84.shp'
    )
    geo_df = geo_df[geo_df.Name_new == _fieldname]
    geo_df2 = geo_df.buffer(-0.00004, resolution=1)
    # put Lon, Lat from dataframe to GeoDataFrame
    geometry = [Point(xy) for xy in zip(group.Longitude, group.Latitude)]
    crs = {'init': 'epsg:4326'}
    # schneide group mit felddaten
    gdf = gpd.GeoDataFrame(group, crs=crs, geometry=geometry)
    mask = gdf.geometry.within(geo_df2.geometry.unary_union)
    newdata = gdf.loc[mask]
    # ertrag cut als einzelwert fürs feld schreiben
    group['Ertrag_wBorders'] = newdata['Ertr.masse (Nass)(tonne/ha)'].sum(
    ) * conversion / newdata['Ertr.masse (Nass)(tonne/ha)'].shape[0]
    return group


yields_df = yields_df.groupby(['Name']).apply(getYieldwithoutBorders)
yields_df.reset_index(drop=True, inplace=True)

In [9]:
# helper functions from https://github.com/ADA-research/AutoML4HybridEarthScienceModels
def extract_date_from_url(url):
    return url[url.find("TIME=") + 5:url.find("TIME=") + 15]


def map_to_degrees(x):
    return x + 360 if x < 0 else x


def filter_by_2std(mean, std, target, data):
    upper_limit = mean + 2 * std
    lower_limit = mean - 2 * std
    return data[(data[target] < upper_limit) & (data[target] > lower_limit)]


def drop_unnamed_columns(df):
    return df.loc[:, ~df.columns.str.contains('^Unnamed')]


def create_pixelwise_S2_data(yields_df, fields, path):
    s2_cols = ["CLM", "dataMask", "B01", "B02", "B03", "B04", "B05", "B06", "B07",
               "B08", "B8A", "B09", "B11", "B12", "solar_azimuth", "solar_zenith",
               "observer_azimuth", "observer_zenith", "unknown"]

    data = []
    for field in tqdm(fields):
        yield_data = yields_df[yields_df["Name"] == field][
            ["Latitude", "Longitude", "Ertr.masse (Nass)(tonne/ha)", "ErtragNass"]]

        for img_dir in os.listdir(os.path.join(path, field)):
            try:
                with rio.open(os.path.join(path, field, img_dir, "response.tiff"), mode="r+") as src, \
                        open(os.path.join(path, field, img_dir, "request.json")) as req_file:
                    msg = json.load(req_file)
                    img_date = extract_date_from_url(msg["url"])

                    transformer = Transformer.from_crs(
                        "EPSG:4326", "EPSG:3857", authority="EPSG")
                    yield_data["x"], yield_data["y"] = transformer.transform(
                        yield_data["Latitude"], yield_data["Longitude"])

                    s2_data = list(rio.sample.sample_gen(
                        src, yield_data[["x", "y"]].values))
                    temp_df = pd.DataFrame(s2_data, columns=s2_cols).drop_duplicates().join(
                        yield_data.reset_index())
                    temp_df["relative_azimuth"] = (
                        temp_df["solar_azimuth"] - temp_df["observer_azimuth"]).apply(map_to_degrees)
                    temp_df["date"] = img_date
                    temp_df["Name"] = field
                    data.append(temp_df)
            except Exception as e:
                print(
                    f"Error processing {os.path.join(path, field, img_dir, 'response.tiff')}: {e}")

    return pd.concat(data)


def resample_and_merge_data(sat_df, et0_df, frequency="W"):
    sat_df["date"] = pd.to_datetime(sat_df["date"])
    et0_df["date"] = pd.to_datetime(et0_df["date"])

    sat_df = sat_df[sat_df["CLM"] == 0]
    sat_df = sat_df.groupby("index").resample(
        frequency, on="date").mean().interpolate().reset_index("date")
    et0_df = et0_df[["date", "et0", "rain", "cum_rain"]].drop_duplicates()
    et0_df = et0_df.resample(frequency, on="date", origin=sat_df["date"].min(
    )).mean().interpolate().reset_index("date")

    return drop_unnamed_columns(sat_df.merge(et0_df, on="date"))


def invert_rtm(rtm_df, model, hyperparams, feature_cols, target_col="lai", do_cv=True):
    pipeline = Pipeline([('scaler', StandardScaler()),
                        ('model', model(**hyperparams))])
    if do_cv:
        results = cross_validate(pipeline, X=rtm_df[feature_cols], y=rtm_df[target_col],
                                 cv=5, scoring=('r2', 'neg_mean_squared_error'), return_train_score=True)
        print(f"Inversion for {target_col}")
        print(
            f"Mean train R2: {np.mean(results['train_r2'])}, individual folds: {results['train_r2']}")
        print(
            f"Mean test R2: {np.mean(results['test_r2'])}, individual folds: {results['test_r2']}\n")

    pipeline.fit(rtm_df[feature_cols], rtm_df[target_col])
    return pipeline


def create_dataset(bands,
                   yields_df,
                   fields,
                   should_create_files=True,
                   include_rtm=False,
                   frequency="W"):
    data_path = "."
    if should_create_files:
        # To create locally:

        sat_images_path = "../../data/cropdata/Bavaria/yields/sat_images_10m/"

        # yields_df = pd.read_csv(os.path.join(data_path, "../datayields2018.csv"))
        fields_of_interest = fields

        sat_df = create_pixelwise_S2_data(yields_df, fields_of_interest,
                                          sat_images_path)
        # S2 values are scaled by a factor 10000
        sat_df[bands] = sat_df[bands] / 10000
        et0_df = pd.read_excel(
            os.path.join(
                "../../data/cropdata/Bavaria/yields/satellite_data_orginal.xlsx"
            ))

        df = resample_and_merge_data(sat_df, et0_df, frequency)

    else:
        # To simply load files that were already created:
        filename = "reflectance_per_pixel_weekly_10m_rtm.csv" \
            if frequency == "W" else "reflectance_per_pixel_monthly_10m_rtm.csv"
        df = pd.read_csv(os.path.join(data_path, filename))

    if include_rtm:
        # For now use a similar simple model setup for RTM inversion
        rf = RandomForestRegressor
        hyperparams = {
            "n_jobs": -1,
            "n_estimators": 300,
            "max_depth": 100,
            "max_features": 'sqrt',
            "random_state": 984
        }

        include_angles = True

        angles = ['solar_zenith', 'observer_zenith', 'relative_azimuth']
        features = bands + angles if include_angles else bands

        lai_model = invert_rtm(rtm_df,
                               rf,
                               hyperparams,
                               feature_cols=features,
                               target_col="lai")
        cm_model = invert_rtm(rtm_df,
                              rf,
                              hyperparams,
                              feature_cols=features,
                              target_col="cm")
        cab_model = invert_rtm(rtm_df,
                               rf,
                               hyperparams,
                               feature_cols=features,
                               target_col="cab")

        df["lai"] = lai_model.predict(df[features])
        df["cm"] = cm_model.predict(df[features])
        df["cab"] = cab_model.predict(df[features])

    return df


def flatten_time_series(df, feature_cols, target_col):
    """
    Flattens a dataset for use in a supervised model. Not suitable for recurrent models.

    Args:
        df (pd DataFrame):
        feature_cols (list of str): Feature column names
        target_col (str):

    return
        df (pd DataFrame):
        feature_cols (list of str): New feature column names with 
                                    suffix for each timestep, 
                                    e.g. _t-5 for 5 weeks/months before last timestep
    """

    out_df = []
    for field_index in df["index"].unique():
        sub_df = df[df["index"] == field_index]
        n_timesteps = len(sub_df)
        cols = list(
            np.array([[col + "_t-{}".format(i) for col in feature_cols]
                      for i in reversed(range(n_timesteps))]).flatten())
        ts_df = pd.DataFrame(sub_df[feature_cols].values.flatten()).T
        ts_df.columns = cols
        ts_df[target_col] = sub_df.iloc[0][target_col]
        out_df.append(ts_df)
    return pd.concat(out_df).interpolate(), cols

In [12]:


def create_dataset(bands,
                   yields_df,
                   fields,
                   should_create_files=True,
                   include_rtm=False,
                   frequency="W"):
    data_path = "."
    if should_create_files:
        # To create locally:

        sat_images_path = "../../data/cropdata/Bavaria/yields/sat_images_10m/"

        # yields_df = pd.read_csv(os.path.join(data_path, "../datayields2018.csv"))
        fields_of_interest = fields

        sat_df = create_pixelwise_S2_data(yields_df, fields_of_interest,
                                          sat_images_path)
        # S2 values are scaled by a factor 10000
        sat_df[bands] = sat_df[bands] / 10000
        et0_df = pd.read_excel(
            os.path.join(
                "../../data/cropdata/Bavaria/yields/satellite_data_orginal.xlsx"
            ))

        df = resample_and_merge_data(sat_df, et0_df, frequency)

    else:
        # To simply load files that were already created:
        filename = "reflectance_per_pixel_weekly_10m_rtm.csv" \
            if frequency == "W" else "reflectance_per_pixel_monthly_10m_rtm.csv"
        df = pd.read_csv(os.path.join(data_path, filename))

    if include_rtm:
        # For now use a similar simple model setup for RTM inversion
        rf = RandomForestRegressor
        hyperparams = {
            "n_jobs": -1,
            "n_estimators": 300,
            "max_depth": 100,
            "max_features": 'sqrt',
            "random_state": 984
        }

        include_angles = True

        angles = ['solar_zenith', 'observer_zenith', 'relative_azimuth']
        features = bands + angles if include_angles else bands

        lai_model = invert_rtm(rtm_df,
                               rf,
                               hyperparams,
                               feature_cols=features,
                               target_col="lai")
        cm_model = invert_rtm(rtm_df,
                              rf,
                              hyperparams,
                              feature_cols=features,
                              target_col="cm")
        cab_model = invert_rtm(rtm_df,
                               rf,
                               hyperparams,
                               feature_cols=features,
                               target_col="cab")

        df["lai"] = lai_model.predict(df[features])
        df["cm"] = cm_model.predict(df[features])
        df["cab"] = cab_model.predict(df[features])

    return df

In [13]:
df = create_dataset(bands=bands, yields_df=yields_df, fields=fields)
out_df, feature_cols = flatten_time_series(df, feature_cols,
                                           "Ertr.masse (Nass)(tonne/ha)")

mean = out_df['Ertr.masse (Nass)(tonne/ha)'].mean()
std = out_df['Ertr.masse (Nass)(tonne/ha)'].std()

# out_df = filter_by_2std(mean, std,'Ertr.masse (Nass)(tonne/ha)', out_df )

100%|██████████| 31/31 [02:53<00:00,  5.58s/it]


In [14]:
out_df.describe()

,B04_t-20,B05_t-20,B06_t-20,B07_t-20,B08_t-20,B8A_t-20,B09_t-20,B11_t-20,B12_t-20,et0_t-20,...,B07_t-0,B08_t-0,B8A_t-0,B09_t-0,B11_t-0,B12_t-0,et0_t-0,rain_t-0,cum_rain_t-0,Ertr.masse (Nass)(tonne/ha)
count,12340.000000,12340.000000,12340.000000,12340.000000,12340.000000,12340.000000,12340.000000,12340.000000,12340.000000,12340.000000,...,12340.000000,12340.000000,12340.000000,12340.000000,12340.000000,12340.000000,1.234000e+04,1.234000e+04,12340.000,12340.000000
mean,0.000008,0.000013,0.000018,0.000020,0.000021,0.000021,0.000021,0.000017,0.000012,1.298619,...,0.000020,0.000020,0.000022,0.000023,0.000025,0.000019,4.420460e+00,3.440000e-01,321.342,6.642245
std,0.000003,0.000004,0.000007,0.000008,0.000008,0.000008,0.000008,0.000006,0.000004,0.000000,...,0.000006,0.000006,0.000007,0.000007,0.000009,0.000007,8.882144e-16,5.551340e-17,0.000,2.099927
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.298619,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.420460e+00,3.440000e-01,321.342,0.286000
25%,0.000007,0.000011,0.000015,0.000016,0.000017,0.000017,0.000018,0.000016,0.000011,1.298619,...,0.000015,0.000016,0.000017,0.000019,0.000020,0.000015,4.420460e+00,3.440000e-01,321.342,5.253000
50%,0.000009,0.000013,0.000018,0.000019,0.000021,0.000020,0.000021,0.000018,0.000013,1.298619,...,0.000021,0.000021,0.000022,0.000023,0.000024,0.000018,4.420460e+00,3.440000e-01,321.342,6.691000
75%,0.000010,0.000015,0.000022,0.000024,0.000025,0.000025,0.000026,0.000020,0.000014,1.298619,...,0.000024,0.000024,0.000026,0.000027,0.000030,0.000026,4.420460e+00,3.440000e-01,321.342,8.143000
max,0.000047,0.000039,0.000045,0.000048,0.000052,0.000050,0.000046,0.000034,0.000029,1.298619,...,0.000048,0.000046,0.000049,0.000047,0.000047,0.000035,4.420460e+00,3.440000e-01,321.342,17.240000


In [16]:
out_df[feature_cols]
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(out_df[feature_cols].values)

scaler = preprocessing.StandardScaler()
scaler.fit(out_df[feature_cols].values)
x_scaled2 = scaler.transform(out_df[feature_cols].values)

test = pd.DataFrame(columns=feature_cols, data=x_scaled2)

In [17]:
out_df = filter_by_2std(mean, std, 'Ertr.masse (Nass)(tonne/ha)', out_df)

In [18]:
# Train RF with pixels and apply
cv = ShuffleSplit(n_splits=5, test_size=0.5, random_state=0)
rf = RandomForestRegressor(n_jobs=4, n_estimators=100)

target_col = "Ertr.masse (Nass)(tonne/ha)"
# feature_cols = list(range(0, 326))

results = cross_validate(rf,
                         X=out_df[feature_cols],
                         y=out_df[target_col],
                         cv=cv,
                         scoring=('r2', 'neg_mean_squared_error'),
                         return_train_score=True)

display("Mean train R2: {}, individual folds: {}".format(
    np.mean(results["train_r2"]), results["train_r2"]))
display("Mean test R2: {}, individual folds: {}".format(
    np.mean(results["test_r2"]), results["test_r2"]))

'Mean train R2: 0.9430185736708638, individual folds: [0.94320968 0.94251857 0.94098539 0.94596446 0.94241477]'

'Mean test R2: 0.5900296355296978, individual folds: [0.59658498 0.59257612 0.59670335 0.57854101 0.58574272]'